# Sales Data Preprocessing
This notebook handles the data cleaning and preprocessing for the sales dashboard project.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

# Create output directory if it doesn't exist
os.makedirs('../data/processed', exist_ok=True)

In [2]:
def load_data():
    # Load raw data
    sales_df = pd.read_csv('../data/raw/sales_data.csv')
    customers_df = pd.read_csv('../data/raw/customers.csv')
    products_df = pd.read_csv('../data/raw/products.csv')
    
    return sales_df, customers_df, products_df

In [3]:
def clean_data(sales_df, customers_df, products_df):
    # Convert date columns to datetime
    sales_df['OrderDate'] = pd.to_datetime(sales_df['OrderDate'])
    customers_df['JoinDate'] = pd.to_datetime(customers_df['JoinDate'])
    customers_df['LastPurchaseDate'] = pd.to_datetime(customers_df['LastPurchaseDate'])
    
    # Handle missing values
    sales_df = sales_df.dropna(subset=['CustomerID', 'ProductID'])
    customers_df = customers_df.drop_duplicates(subset=['CustomerID'])
    products_df = products_df.drop_duplicates(subset=['ProductID'])
    
    # Calculate total price
    sales_df['TotalPrice'] = sales_df['Quantity'] * sales_df['UnitPrice']
    
    return sales_df, customers_df, products_df

In [4]:
def calculate_rfm(sales_df, customers_df, current_date=None):
    if current_date is None:
        current_date = sales_df['OrderDate'].max()
    
    # Calculate RFM metrics
    rfm = sales_df.groupby('CustomerID').agg({
        'OrderDate': lambda x: (current_date - x.max()).days,  # Recency
        'OrderID': 'count',  # Frequency
        'TotalPrice': 'sum'  # Monetary
    }).reset_index()
    
    # Rename columns
    rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']
    
    # Merge with customer data
    rfm = pd.merge(rfm, customers_df, on='CustomerID', how='left')
    
    return rfm

In [5]:
def main():
    # Load data
    sales_df, customers_df, products_df = load_data()
    
    # Clean data
    sales_df, customers_df, products_df = clean_data(sales_df, customers_df, products_df)
    
    # Calculate RFM
    rfm_df = calculate_rfm(sales_df, customers_df)
    
    # Save processed data
    sales_df.to_csv('../data/processed/processed_sales.csv', index=False)
    customers_df.to_csv('../data/processed/processed_customers.csv', index=False)
    products_df.to_csv('../data/processed/processed_products.csv', index=False)
    rfm_df.to_csv('../data/processed/customer_rfm.csv', index=False)
    
    print("Data processing complete!")

In [6]:
if __name__ == "__main__":
    main()

Data processing complete!
